# Mistral with Azure AI Foundry

> https://learn.microsoft.com/en-us/azure/machine-learning/concept-models-featured?view=azureml-api-2#mistral-ai

In [1]:
import gradio as gr
import os
import requests
import sys

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage, AssistantMessage
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError
from datetime import datetime
from dotenv import load_dotenv

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 18-Apr-2025 07:36:00


In [4]:
load_dotenv("azure.env")

True

In [5]:
model = "mistral-small-2503"

"mistral-small-2503"
- Input: text and images (131,072 tokens),
image-based tokens are 16px x 16px
blocks of the original images
- Output: text (4,096 tokens)
- Tool calling: Yes
- Response formats: Text, JSON

In [6]:
def ask_mistral(prompt: str) -> str:
    """
    Sends a prompt to the Mistral model via Azure's ChatCompletionsClient and returns the response.

    Args:
        prompt (str): The user's input prompt to be processed by the AI assistant.

    Returns:
        str: The AI-generated response from the Mistral model. If an error occurs, 
             a fallback error message is returned.
    """
    try:
        client = ChatCompletionsClient(endpoint=os.getenv("endpoint"),
                                       credential=AzureKeyCredential(os.getenv("key")))

        response = client.complete(
            messages=[
                SystemMessage(content="You are a helpful AI assistant."),
                UserMessage(content=prompt),
            ],
            model=model,
            max_tokens=1000,
        )
        return response.choices[0].message.content.strip()

    except Exception as e:
        print(f"Error during completion: {e}")
        return "Sorry, something went wrong while processing your request."

In [7]:
prompt = "What is the capital of France?"

print(ask_mistral(prompt))

The capital of France is Paris. It is known for iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral, as well as its rich history, culture, and cuisine.


In [8]:
prompt = "What are 3 things to visit in Paris?"

print(ask_mistral(prompt))

Paris, the City of Light, offers a wealth of iconic sites to visit. Here are three must-see attractions:

1. **Eiffel Tower**:
   - **Description**: The most famous symbol of Paris, the Eiffel Tower offers breathtaking views of the city from its three levels. You can dine at one of its restaurants or simply enjoy the panoramic vista.
   - **Location**: Champ de Mars, 5 Avenue Anatole France, 75007 Paris

2. **Louvre Museum**:
   - **Description**: Home to thousands of works of art, including the Mona Lisa and the Winged Victory, the Louvre is the world's largest and most visited art museum. The glass pyramid in the courtyard is also a notable landmark.
   - **Location**: Rue de Rivoli, 75001 Paris

3. **Notre-Dame Cathedral**:
   - **Description**: Although severely damaged by a fire in 2019, Notre-Dame remains an iconic symbol of Paris. The cathedral's Gothic architecture and historical significance make it a must-visit. Restoration work is ongoing, but you can still admire its exteri

## Webapp

In [9]:
def mistral_chat(user_input, history):

    try:
        client = ChatCompletionsClient(endpoint=os.getenv("endpoint"),
                                       credential=AzureKeyCredential(os.getenv("key")))

        # Convert previous messages to Azure message objects
        messages = [
            SystemMessage(
                content=
                "You are an AI assistant that helps people find information.")
        ]

        for user_msg, assistant_msg in history:
            messages.append(UserMessage(content=user_msg))
            messages.append(AssistantMessage(content=assistant_msg))
            messages.append(UserMessage(content=user_input))
        response = client.complete(
            messages=messages,
            model=model,
        )
        reply = response.choices[0].message.content

    except HttpResponseError as ex:
        if ex.status_code == 400:
            try:
                error_data = json.loads(ex.response._content.decode('utf-8'))
                if isinstance(error_data, dict) and "error" in error_data:
                    reply = f"[ERROR {error_data['error']['code']}] {error_data['error']['message']}"
                else:
                    reply = "An unknown error occurred while processing your request."
            except Exception:
                reply = "Failed to decode error response."
        else:
            reply = f"An unexpected error occurred: {str(ex)}"

    history.append((user_input, reply))

    return history, history

In [10]:
with gr.Blocks() as webapp:
    gr.Markdown("# 🤖 Chat with Mistral in Azure AI Foundry")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your message", placeholder="Your prompt:", lines=1)
    clear = gr.Button("CLEAR")
    state = gr.State([])
    msg.submit(mistral_chat, [msg, state], [chatbot, state])
    clear.click(lambda: ([], []), None, [chatbot, state])


webapp.launch(share=True)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://cc0ecbe0902243c16e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
